<a href="https://colab.research.google.com/github/nura6199/my-dissertation/blob/main/hybrid_ensemble_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

upload the dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving chronic_kidney_disease.csv to chronic_kidney_disease.csv


In [3]:
# Load dataset
data = pd.read_csv('chronic_kidney_disease.csv')


In [4]:
# Convert categorical data into numerical values using one-hot encoding
#data = pd.get_dummies(data)
#Convert categorical data into numerical values
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == 'object':
        data[column] = le.fit_transform(data[column].astype(str))


In [9]:
# Preprocessing
data = data.drop(['age', 'appet', 'ane', 'pe', 'ba'], axis=1) # remove irrelevant features and categorical features
data = data.dropna() # remove missing values


In [10]:
data.sample(20)

,bp,sg,al,su,rbc,pc,pcc,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,class
392,8,3,0,0,0,0,1,32,75,8,30,17,48,46,6600,5.5,1,1,1,1
231,9,5,6,6,0,0,1,94,79,44,21,10,15,35,0,0.0,0,0,0,0
260,8,3,0,0,0,0,1,30,65,6,30,11,41,45,9400,5.3,1,1,1,1
395,8,3,0,0,0,0,1,37,76,1,31,23,57,47,6700,4.9,1,1,1,1
184,6,2,3,2,0,1,1,109,13,50,16,19,13,31,5800,3.6,0,0,0,0
172,8,1,1,2,0,0,1,106,6,45,13,0,6,34,12800,4.9,1,1,1,0
202,6,5,6,6,0,0,1,14,98,45,18,34,96,24,0,0.0,1,0,1,0
183,7,2,0,0,0,0,1,1,3,70,18,17,115,0,0,0.0,1,1,1,0
83,7,2,1,0,0,0,1,26,35,6,17,9,115,0,0,0.0,0,0,1,0
42,0,1,0,0,0,0,1,63,54,6,22,16,112,33,9200,4.5,0,1,1,0


In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=42)

In [14]:
data.head()

,bp,sg,al,su,rbc,pc,pcc,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,class
0,8,3,1,0,0,0,1,21,63,8,34,40,54,44,7800,5.2,0,0,1,0
1,5,3,4,0,0,0,1,146,32,4,34,40,13,38,6000,0.0,1,1,1,0
2,8,1,2,3,0,0,1,114,81,14,34,40,111,31,7500,0.0,1,0,1,0
3,7,0,4,0,0,1,0,17,84,53,1,0,12,32,6700,3.9,0,1,1,0
4,8,1,2,0,0,0,1,6,51,10,34,40,16,35,7300,4.6,1,1,1,0


Returns the number of missing values in the dataset.

In [15]:
data.isnull().sum()

bp       0
sg       0
al       0
su       0
rbc      0
pc       0
pcc      0
bgr      0
bu       0
sc       0
sod      0
pot      0
hemo     0
pcv      0
wbcc     0
rbcc     0
htn      0
dm       0
cad      0
class    0
dtype: int64

In [16]:
# Drop columns with missing values
data.dropna(inplace=True)

In [17]:
data.describe()

,bp,sg,al,su,rbc,pc,pcc,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc,htn,dm,cad,class
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.0000,400.000000,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000,400.000000
mean,6.930000,2.777500,1.590000,1.130000,0.117500,0.190000,0.885000,73.612500,61.582500,24.532500,23.842500,22.5450,58.070000,31.982500,6178.500000,3.16575,0.627500,0.645000,0.905000,0.375000
std,2.186791,1.346301,2.037715,2.092617,0.322418,0.392792,0.319421,53.208083,28.950276,25.634875,7.739343,11.5595,37.602926,16.962799,4490.489839,2.36621,0.484076,0.479113,0.293582,0.484729
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,6.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,23.750000,44.000000,5.000000,18.000000,14.0000,26.000000,26.000000,0.000000,0.00000,0.000000,0.000000,1.000000,0.000000
50%,7.000000,3.000000,1.000000,0.000000,0.000000,0.000000,1.000000,64.500000,63.000000,10.000000,23.000000,21.0000,50.000000,37.000000,6900.000000,4.00000,1.000000,1.000000,1.000000,0.000000
75%,8.000000,4.000000,3.000000,1.000000,0.000000,0.000000,1.000000,131.250000,77.000000,43.000000,31.000000,32.2500,96.250000,44.000000,9400.000000,5.10000,1.000000,1.000000,1.000000,1.000000
max,10.000000,5.000000,6.000000,6.000000,1.000000,1.000000,1.000000,146.000000,118.000000,84.000000,34.000000,40.0000,115.000000,54.000000,26400.000000,8.00000,1.000000,1.000000,1.000000,1.000000


In [18]:
# Initialize classifiers
nb_clf = GaussianNB()
dt_clf = DecisionTreeClassifier(random_state=42)
svm_clf = SVC(kernel='rbf', gamma='auto')

In [19]:
# Train classifiers on training set
nb_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)

SVC(gamma='auto')

In [20]:
# Make predictions on testing set
nb_pred = nb_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
svm_pred = svm_clf.predict(X_test)

In [21]:
# Combine predictions using majority voting
ensemble_pred = []
for i in range(len(X_test)):
    preds = [nb_pred[i], dt_pred[i], svm_pred[i]]
    counts = {pred: preds.count(pred) for pred in preds}
    majority_pred = max(counts, key=counts.get)
    ensemble_pred.append(majority_pred)

In [22]:
# Evaluate the performance of the ensemble model using different metrics
acc = accuracy_score(y_test, ensemble_pred)
f1 = f1_score(y_test, ensemble_pred, average='weighted')
recall = recall_score(y_test, ensemble_pred, average='weighted')
precision = precision_score(y_test, ensemble_pred, average='weighted')

print(f"Accuracy: {acc}")
print(f"F1 Score: {f1}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")

Accuracy: 0.975
F1 Score: 0.9747728860936409
Recall: 0.975
Precision: 0.975925925925926
